# All Tags Above 10 Samples - Balanced Classification Experiment
## Balanced dataset with ALL tags that have >10 samples

This experiment uses ALL tags with >10 samples (not just top 10) to get comprehensive results.
All classes are balanced to equal representation to remove class imbalance bias.

In [3]:
!nvidia-smi | grep "CUDA Version"
!pip install pandas numpy faiss-gpu-cu12 scikit-learn matplotlib seaborn tqdm sentence-transformers


| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 MB 8.5 MB/s  0:00:05m0:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 139.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 129.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 93.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.2 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2━━━━━━ 0/8 [numpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [seaborn]m7/8 [seaborn]ib]u12]


In [4]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import faiss
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import json
import warnings
warnings.filterwarnings('ignore')

print("Imports successful!")

Imports successful!


## Configuration

In [5]:
# Paths
ROOT = Path.cwd()
BASE_DIR = ROOT / "gemini25flash"
CSV_PATH = ROOT / "download_status_with_tags.csv"
RESULTS_DIR = ROOT / "results_all_tags_above_10"
RESULTS_DIR.mkdir(exist_ok=True)

print(f"GE Experiments root: {ROOT}")
print(f"Base directory: {BASE_DIR}")
print(f"  Exists: {BASE_DIR.exists()}")
print(f"CSV path: {CSV_PATH}")
print(f"  Exists: {CSV_PATH.exists()}")
print(f"Results directory: {RESULTS_DIR}")

GE Experiments root: /workspace/embeddings
Base directory: /workspace/embeddings/gemini25flash
  Exists: True
CSV path: /workspace/embeddings/download_status_with_tags.csv
  Exists: True
Results directory: /workspace/embeddings/results_all_tags_above_10


## Load and Balance Data

In [6]:
# Load CSV with tags
df = pd.read_csv(CSV_PATH)
df['doc_name'] = df['filename'].str.replace('.pdf', '', regex=False)
unique_docs = df.drop_duplicates(subset=['doc_name'])

print(f"Loaded {len(unique_docs)} unique documents")
print(f"\nOriginal tag distribution:")
tag_counts = unique_docs['document_tag'].value_counts()
print(tag_counts.head(25))

Loaded 564 unique documents

Original tag distribution:
document_tag
ГИР         67
ГЕОДЕЗ      31
АВИА        31
АВЗУ        29
ЭП          27
ОЧ          26
ПИР         26
МИОРЕК      26
ГЕОЛОГ      26
ОПЗ         24
ПЗЗ         23
ЗНП         19
ПОС         18
ЛИЦГЕН      18
ДЕФАКТ      16
ТУЭС        14
ИНФОЗАК     13
ПП          13
АПЗ         12
НАЧСТРОИ    12
ИНФОГЕН     12
ИСТФИН       9
ЛИЦИЗЫСК     8
ТУТМ         8
ДОЗИМЕТР     7
Name: count, dtype: int64


In [7]:
# Select ALL tags with >10 samples (not just top 10)
MIN_SAMPLES = 10

# Get ALL tags with sufficient samples
valid_tags = tag_counts[tag_counts > MIN_SAMPLES].index.tolist()

print(f"Selected {len(valid_tags)} tags with >{MIN_SAMPLES} samples:")
for tag in valid_tags:
    count = tag_counts[tag]
    print(f"  {tag}: {count} samples")

# Filter dataset to only include selected tags
filtered_docs = unique_docs[unique_docs['document_tag'].isin(valid_tags)].copy()
print(f"\nFiltered dataset: {len(filtered_docs)} documents")

Selected 21 tags with >10 samples:
  ГИР: 67 samples
  ГЕОДЕЗ: 31 samples
  АВИА: 31 samples
  АВЗУ: 29 samples
  ЭП: 27 samples
  ОЧ: 26 samples
  ПИР: 26 samples
  МИОРЕК: 26 samples
  ГЕОЛОГ: 26 samples
  ОПЗ: 24 samples
  ПЗЗ: 23 samples
  ЗНП: 19 samples
  ПОС: 18 samples
  ЛИЦГЕН: 18 samples
  ДЕФАКТ: 16 samples
  ТУЭС: 14 samples
  ИНФОЗАК: 13 samples
  ПП: 13 samples
  АПЗ: 12 samples
  НАЧСТРОИ: 12 samples
  ИНФОГЕН: 12 samples

Filtered dataset: 483 documents


In [8]:
# Balance the dataset: use min class size to avoid overrepresentation
# This ensures no class dominates the results
min_class_size = filtered_docs['document_tag'].value_counts().min()

print(f"Minimum class size: {min_class_size}")
print(f"Balancing all {len(valid_tags)} classes to {min_class_size} samples each")

# Sample equal number from each class
balanced_docs = []
np.random.seed(42)  # For reproducibility

for tag in valid_tags:
    tag_docs = filtered_docs[filtered_docs['document_tag'] == tag]
    sampled_docs = tag_docs.sample(n=min_class_size, random_state=42)
    balanced_docs.append(sampled_docs)

balanced_dataset = pd.concat(balanced_docs, ignore_index=True)

print(f"\nBalanced dataset: {len(balanced_dataset)} documents")
print(f"Expected: {len(valid_tags)} tags × {min_class_size} samples = {len(valid_tags) * min_class_size}")
print(f"\nBalanced distribution:")
print(balanced_dataset['document_tag'].value_counts().sort_index())

Minimum class size: 12
Balancing all 21 classes to 12 samples each

Balanced dataset: 252 documents
Expected: 21 tags × 12 samples = 252

Balanced distribution:
document_tag
АВЗУ        12
АВИА        12
АПЗ         12
ГЕОДЕЗ      12
ГЕОЛОГ      12
ГИР         12
ДЕФАКТ      12
ЗНП         12
ИНФОГЕН     12
ИНФОЗАК     12
ЛИЦГЕН      12
МИОРЕК      12
НАЧСТРОИ    12
ОПЗ         12
ОЧ          12
ПЗЗ         12
ПИР         12
ПОС         12
ПП          12
ТУЭС        12
ЭП          12
Name: count, dtype: int64


## Initialize Services

In [10]:
from text_selector import TextSelector

# Initialize text selector
text_selector = TextSelector(BASE_DIR)
print("Text selector initialized")


Text selector initialized


## Define Experiment Strategies

Same strategies as original experiment for direct comparison

In [13]:
# Define strategies - same as original for comparison
strategies = [
    # First N pages
    {'name': 'first_1_page', 'config': {'type': 'first_n', 'n': 1}},
    {'name': 'first_2_pages', 'config': {'type': 'first_n', 'n': 2}},
    {'name': 'first_3_pages', 'config': {'type': 'first_n', 'n': 3}},
    {'name': 'first_5_pages', 'config': {'type': 'first_n', 'n': 5}},
    
    # Last N pages
    {'name': 'last_1_page', 'config': {'type': 'last_n', 'n': 1}},
    {'name': 'last_2_pages', 'config': {'type': 'last_n', 'n': 2}},
    
    # First and last
    {'name': 'first_and_last_1', 'config': {'type': 'first_and_last', 'n': 1}},
    {'name': 'first_and_last_2', 'config': {'type': 'first_and_last', 'n': 2}},
    
    # Middle pages
    {'name': 'middle_3_pages', 'config': {'type': 'middle_n', 'n': 3}},
    {'name': 'middle_5_pages', 'config': {'type': 'middle_n', 'n': 5}},
    
    # Percentage-based
    {'name': 'first_5_percent', 'config': {'type': 'percentage', 'percentage': 0.05, 'position': 'start'}},
    {'name': 'first_10_percent', 'config': {'type': 'percentage', 'percentage': 0.10, 'position': 'start'}},
    {'name': 'middle_5_percent', 'config': {'type': 'percentage', 'percentage': 0.05, 'position': 'middle'}},
    
    # Character-based
    {'name': 'first_1000_chars', 'config': {'type': 'first_chars', 'n_chars': 1000}},
    {'name': 'first_2000_chars', 'config': {'type': 'first_chars', 'n_chars': 2000}},
    {'name': 'first_5000_chars', 'config': {'type': 'first_chars', 'n_chars': 5000}},
    
    # Baseline
    {'name': 'all_pages', 'config': {'type': 'all'}},
]

print(f"Defined {len(strategies)} strategies to test")

Defined 17 strategies to test


In [15]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("google/embeddinggemma-300m", trust_remote_code=True)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: google/embeddinggemma-300m
INFO:sentence_transformers.SentenceTransformer:14 prompts are loaded, with the keys: ['query', 'document', 'BitextMining', 'Clustering', 'Classification', 'InstructionRetrieval', 'MultilabelClassification', 'PairClassification', 'Reranking', 'Retrieval', 'Retrieval-query', 'Retrieval-document', 'STS', 'Summarization']


## Helper Functions

In [17]:
def extract_texts_for_strategy(df, strategy_config):
    """
    Extract texts for all documents using a given strategy.
    
    Returns:
        List of (doc_name, text, tag, file_type) tuples
    """
    results = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting texts"):
        doc_name = row['doc_name']
        tag = row['document_tag']
        file_type = row['file_type']
        
        try:
            text = text_selector.select_by_strategy(doc_name, strategy_config)
            if text.strip():
                results.append((doc_name, text, tag, file_type))
            else:
                print(f"  Warning: Empty text for {doc_name}")
        except Exception as e:
            print(f"  Error processing {doc_name}: {e}")
    
    return results


def embed_texts(texts, batch_size=32):
    """
    Embed texts using SentenceTransformer model.
    
    Args:
        texts: List of strings to embed
        model: SentenceTransformer model instance
        batch_size: Batch size for encoding
    
    Returns:
        numpy array of embeddings
    """
    if not texts:
        return np.array([])
    
    print(f"Embedding {len(texts)} texts...")
    
    # Use encode_document for document embeddings
    embeddings = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True
    )
    
    print(f"Generated {len(embeddings)} embeddings with shape {embeddings.shape}")
    
    return embeddings


def classify_with_faiss(train_embeddings, train_labels, test_embeddings, k=5):
    """
    Classify using FAISS k-NN with cosine similarity.
    """
    # Normalize for cosine similarity
    faiss.normalize_L2(train_embeddings)
    faiss.normalize_L2(test_embeddings)
    
    # Build FAISS index
    dim = train_embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(train_embeddings)
    
    # Search k nearest neighbors
    distances, indices = index.search(test_embeddings, k)
    
    # Majority vote
    predictions = []
    for neighbor_indices in indices:
        neighbor_labels = train_labels[neighbor_indices]
        unique, counts = np.unique(neighbor_labels, return_counts=True)
        prediction = unique[np.argmax(counts)]
        predictions.append(prediction)
    
    return np.array(predictions)


def calculate_metrics(y_true, y_pred, strategy_name):
    """
    Calculate classification metrics.
    For balanced data, macro-average is more appropriate.
    """
    accuracy = accuracy_score(y_true, y_pred)
    
    # Use macro average for balanced datasets (equal weight to each class)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    
    # Also calculate weighted for comparison
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        y_true, y_pred, average='weighted', zero_division=0
    )
    
    return {
        'strategy': strategy_name,
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'precision_weighted': precision_weighted,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
    }

print("Helper functions defined")

Helper functions defined


## Run Experiments with Stratified Split

Using stratified split to ensure balanced class distribution in train/test sets

In [18]:
# Configuration
np.random.seed(42)
TRAIN_RATIO = 0.8
K_NEIGHBORS = 5

NUM_TAGS = len(valid_tags)

# Calculate baseline accuracy (random guessing)
BASELINE_RANDOM = 1.0 / NUM_TAGS

all_results = []

print(f"Running balanced experiments:")
print(f"  Dataset: {len(balanced_dataset)} documents ({NUM_TAGS} classes × {min_class_size} samples)")
print(f"  Train/test split: {TRAIN_RATIO*100:.0f}% / {(1-TRAIN_RATIO)*100:.0f}%")
print(f"  K-NN k-value: {K_NEIGHBORS}")
print(f"  Baseline (random): {BASELINE_RANDOM*100:.1f}% accuracy")
print(f"  Using stratified split to maintain class balance\n")

Running balanced experiments:
  Dataset: 252 documents (21 classes × 12 samples)
  Train/test split: 80% / 20%
  K-NN k-value: 5
  Baseline (random): 4.8% accuracy
  Using stratified split to maintain class balance



In [19]:
# Run experiments
for strategy in strategies:
    strategy_name = strategy['name']
    strategy_config = strategy['config']
    
    print(f"\n{'='*80}")
    print(f"Strategy: {strategy_name}")
    print(f"Config: {strategy_config}")
    print(f"{'='*80}")
    
    try:
        # Extract texts
        print("\n1. Extracting texts...")
        text_data = extract_texts_for_strategy(balanced_dataset, strategy_config)
        
        if len(text_data) == 0:
            print(f"  ✗ No texts extracted")
            continue
        
        print(f"  ✓ Extracted {len(text_data)} documents")
        
        # Unpack data
        doc_names = [item[0] for item in text_data]
        texts = [item[1] for item in text_data]
        labels = np.array([item[2] for item in text_data])
        file_types = [item[3] for item in text_data]
        
        # Generate embeddings
        print("\n2. Generating embeddings...")
        embeddings = embed_texts(texts)
        
        if len(embeddings) == 0:
            print(f"  ✗ No embeddings generated")
            continue
        
        print(f"  ✓ Generated {len(embeddings)} embeddings (shape: {embeddings.shape})")
        
        # Align embeddings and labels
        min_len = min(len(embeddings), len(labels))
        embeddings = embeddings[:min_len]
        labels = labels[:min_len]
        
        # Stratified split
        print("\n3. Stratified train/test split...")
        from sklearn.model_selection import train_test_split
        
        indices = np.arange(len(embeddings))
        train_indices, test_indices = train_test_split(
            indices, 
            test_size=(1-TRAIN_RATIO), 
            stratify=labels,
            random_state=42
        )
        
        train_embeddings = embeddings[train_indices].astype('float32')
        train_labels = labels[train_indices]
        test_embeddings = embeddings[test_indices].astype('float32')
        test_labels = labels[test_indices]
        
        print(f"  Train: {len(train_embeddings)} samples")
        print(f"  Test: {len(test_embeddings)} samples")
        
        # Verify stratification - show distribution
        train_unique, train_counts = np.unique(train_labels, return_counts=True)
        test_unique, test_counts = np.unique(test_labels, return_counts=True)
        print(f"  Train classes: {len(train_unique)}, samples per class: {train_counts}")
        print(f"  Test classes: {len(test_unique)}, samples per class: {test_counts}")
        
        # Classify
        print("\n4. Classifying with FAISS k-NN...")
        predictions = classify_with_faiss(train_embeddings, train_labels, test_embeddings, k=K_NEIGHBORS)
        
        # Calculate metrics
        print("\n5. Calculating metrics...")
        metrics = calculate_metrics(test_labels, predictions, strategy_name)
        all_results.append(metrics)
        
        # Print results
        print(f"\n{'─'*80}")
        print(f"Results for {strategy_name}:")
        print(f"  Accuracy:         {metrics['accuracy']:.4f}")
        print(f"  F1 (macro):       {metrics['f1_macro']:.4f}")
        print(f"  F1 (weighted):    {metrics['f1_weighted']:.4f}")
        print(f"  Precision (macro): {metrics['precision_macro']:.4f}")
        print(f"  Recall (macro):    {metrics['recall_macro']:.4f}")
        print(f"{'─'*80}")
        
        # Classification report
        print("\nPer-class metrics:")
        print(classification_report(test_labels, predictions, zero_division=0))
        
    except Exception as e:
        print(f"\n✗ Error: {e}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n\n{'='*80}")
print(f"Completed {len(all_results)} / {len(strategies)} experiments")
print(f"{'='*80}")


Strategy: first_1_page
Config: {'type': 'first_n', 'n': 1}

1. Extracting texts...


Extracting texts:   0%|          | 0/252 [00:00<?, ?it/s]

  Error processing Разрешение местных исполнительных органов на использование земельного участка (площадки, трассы) или правоустанавливающий документ на земельный участок_3: [Errno 36] File name too long: '/workspace/embeddings/gemini25flash/Разрешение местных исполнительных органов на использование земельного участка (площадки, трассы) или правоустанавливающий документ на земельный участок_3'


  Error processing Разрешение местных исполнительных органов на использование земельного участка (площади, трассы) или правоустанавливающий документ на земельный участок_8: [Errno 36] File name too long: '/workspace/embeddings/gemini25flash/Разрешение местных исполнительных органов на использование земельного участка (площади, трассы) или правоустанавливающий документ на земельный участок_8'
  Error processing Разрешение местных исполнительных органов на использование земельного участка (площади, трассы) или правоустанавливающий документ на земельный участок_9: [Errno 36] File name too long: '/workspace/embeddings/gemini25flash/Разрешение местных исполнительных органов на использование земельного участка (площади, трассы) или правоустанавливающий документ на земельный участок_9'
  Error processing Разрешение местных исполнительных органов на использование земельного участка (площади, трассы) или правоустанавливающий документ на земельный участок_1: [Errno 36] File name too long: '/work

  Error processing Лицензия генеральной проектной и субпроектной организаций с приложениями к ней и указанием категории лицензиата, дающих право на данный вид проектной деятельности_1: [Errno 36] File name too long: '/workspace/embeddings/gemini25flash/Лицензия генеральной проектной и субпроектной организаций с приложениями к ней и указанием категории лицензиата, дающих право на данный вид проектной деятельности_1'


...

  Error processing Разрешение местных исполнительных органов на использование земельного участка (площадки, трассы) или правоустанавливающий документ на земельный участок_3: [Errno 36] File name too long: '/workspace/embeddings/gemini25flash/Разрешение местных исполнительных органов на использование земельного участка (площадки, трассы) или правоустанавливающий документ на земельный участок_3'
  Error processing Разрешение местных исполнительных органов на использование земельного участка (площади, трассы) или правоустанавливающий документ на земельный участок_8: [Errno 36] File name too long: '/workspace/embeddings/gemini25flash/Разрешение местных исполнительных органов на использование земельного участка (площади, трассы) или правоустанавливающий документ на земельный участок_8'
  Error processing Разрешение местных исполнительных органов на использование земельного участка (площади, трассы) или правоустанавливающий документ на земельный участок_9: [Errno 36] File name too long: '/wo

  Error processing Лицензия генеральной проектной и субпроектной организаций с приложениями к ней и указанием категории лицензиата, дающих право на данный вид проектной деятельности_1: [Errno 36] File name too long: '/workspace/embeddings/gemini25flash/Лицензия генеральной проектной и субпроектной организаций с приложениями к ней и указанием категории лицензиата, дающих право на данный вид проектной деятельности_1'


  ✓ Extracted 222 documents

2. Generating embeddings...
Embedding 222 texts...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Generated 222 embeddings with shape (222, 768)
  ✓ Generated 222 embeddings (shape: (222, 768))

3. Stratified train/test split...
  Train: 177 samples
  Test: 45 samples
  Train classes: 21, samples per class: [ 3 10  9  9  9  9  9 10  9 10  6  9  6  9  9  4  8 10 10  9 10]
  Test classes: 21, samples per class: [1 2 2 3 2 2 3 2 3 2 2 3 1 3 2 1 2 2 2 3 2]

4. Classifying with FAISS k-NN...

5. Calculating metrics...

────────────────────────────────────────────────────────────────────────────────
Results for all_pages:
  Accuracy:         0.4667
  F1 (macro):       0.3955
  F1 (weighted):    0.4346
  Precision (macro): 0.4263
  Recall (macro):    0.4365
────────────────────────────────────────────────────────────────────────────────

Per-class metrics:
              precision    recall  f1-score   support

        АВЗУ       0.50      1.00      0.67         1
        АВИА       1.00      1.00      1.00         2
         АПЗ       0.17      0.50      0.25         2
      ГЕОДЕЗ       

## Results Summary

In [20]:
# Create results dataframe
results_df = pd.DataFrame(all_results)
results_df = results_df.sort_values('f1_macro', ascending=False)

print("\nAll Results (sorted by F1 macro):")
print(results_df.to_string(index=False))

# Save to CSV
results_csv = RESULTS_DIR / "classification_results_all_tags.csv"
results_df.to_csv(resul5+ts_csv, index=False)
print(f"\nResults saved to: {results_csv}")


All Results (sorted by F1 macro):
        strategy  accuracy  precision_macro  recall_macro  f1_macro  precision_weighted  recall_weighted  f1_weighted
first_2000_chars  0.590909         0.540476      0.563492  0.523016            0.586364         0.590909     0.560227
first_5000_chars  0.522727         0.457937      0.507937  0.462585            0.476894         0.522727     0.478571
first_10_percent  0.522727         0.503175      0.500000  0.458163            0.552273         0.522727     0.496429
first_and_last_2  0.488889         0.485450      0.468254  0.445695            0.527160         0.488889     0.475488
 first_5_percent  0.522727         0.480952      0.492063  0.443991            0.553788         0.522727     0.494264
  middle_5_pages  0.488889         0.439342      0.468254  0.433711            0.471164         0.488889     0.462998
    first_1_page  0.500000         0.426190      0.476190  0.422676            0.465530         0.500000     0.456277
    last_2_pages  0.4

## Visualizations

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle(f'Balanced Dataset ({NUM_TAGS} Classes): Classification Metrics Comparison', fontsize=16, fontweight='bold')

metrics_to_plot = ['accuracy', 'f1_macro', 'f1_weighted', 'precision_macro']
titles = ['Accuracy', 'F1 Score (Macro)', 'F1 Score (Weighted)', 'Precision (Macro)']

for idx, (metric, title) in enumerate(zip(metrics_to_plot, titles)):
    ax = axes[idx // 2, idx % 2]
    
    sorted_df = results_df.sort_values(metric, ascending=True)
    
    bars = ax.barh(sorted_df['strategy'], sorted_df[metric])
    colors = plt.cm.RdYlGn(sorted_df[metric])
    for bar, color in zip(bars, colors):
        bar.set_color(color)
    
    # Add baseline line for accuracy
    if metric == 'accuracy':
        ax.axvline(x=BASELINE_RANDOM, color='red', linestyle='--', linewidth=2, 
                   label=f'Baseline (random): {BASELINE_RANDOM*100:.1f}%')
        ax.legend()
    
    ax.set_xlabel(title, fontsize=12, fontweight='bold')
    ax.set_ylabel('Strategy', fontsize=12, fontweight='bold')
    ax.set_title(f'{title} by Strategy', fontsize=13, fontweight='bold')
    ax.set_xlim(0, 1.0)
    ax.grid(axis='x', alpha=0.3)
    
    for i, (strategy, value) in enumerate(zip(sorted_df['strategy'], sorted_df[metric])):
        ax.text(value + 0.01, i, f'{value:.3f}', va='center', fontsize=9)

plt.tight_layout()
plot_path = RESULTS_DIR / "metrics_comparison_all_tags.png"
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"Plot saved to: {plot_path}")
plt.show()

In [ ]:
# Best strategy
best_strategy = results_df.iloc[0]

print("\n" + "="*80)
print(f"BEST STRATEGY (Balanced Data - {NUM_TAGS} Classes)")
print("="*80)
print(f"\nStrategy: {best_strategy['strategy']}")
print(f"\nMetrics:")
print(f"  Accuracy:          {best_strategy['accuracy']:.4f}")
print(f"  F1 (macro):        {best_strategy['f1_macro']:.4f}")
print(f"  F1 (weighted):     {best_strategy['f1_weighted']:.4f}")
print(f"  Precision (macro): {best_strategy['precision_macro']:.4f}")
print(f"  Recall (macro):    {best_strategy['recall_macro']:.4f}")
print(f"\nImprovement over baseline: {(best_strategy['accuracy'] - BASELINE_RANDOM) / BASELINE_RANDOM * 100:.1f}%")
print("\n" + "="*80)

## Top 5 Strategies

In [ ]:
top_5 = results_df.head(5)

print("\nTop 5 Strategies by F1 Score (Macro):")
print("="*80)
for idx, row in enumerate(top_5.itertuples(), 1):
    print(f"\n{idx}. {row.strategy}")
    print(f"   Accuracy:          {row.accuracy:.4f}")
    print(f"   F1 (macro):        {row.f1_macro:.4f}")
    print(f"   F1 (weighted):     {row.f1_weighted:.4f}")
    print(f"   Precision (macro): {row.precision_macro:.4f}")
    print(f"   Recall (macro):    {row.recall_macro:.4f}")
    print("   " + "-"*76)

## Save Complete Results

In [ ]:
# Save experiment summary
experiment_summary = {
    'configuration': {
        'dataset_type': 'balanced',
        'selection_criteria': 'ALL tags with >10 samples',
        'num_classes': NUM_TAGS,
        'samples_per_class': min_class_size,
        'total_documents': len(balanced_dataset),
        'selected_tags': valid_tags,
        'train_ratio': TRAIN_RATIO,
        'k_neighbors': K_NEIGHBORS,
        'baseline_random_accuracy': BASELINE_RANDOM,
        'stratified_split': True,
        'embedding_service': f"{EMBEDDING_HOST}:{EMBEDDING_PORT}",
    },
    'strategies_tested': len(strategies),
    'successful_experiments': len(all_results),
    'best_strategy': {
        'name': best_strategy['strategy'],
        'accuracy': float(best_strategy['accuracy']),
        'f1_macro': float(best_strategy['f1_macro']),
        'f1_weighted': float(best_strategy['f1_weighted']),
        'precision_macro': float(best_strategy['precision_macro']),
        'recall_macro': float(best_strategy['recall_macro']),
    },
    'all_results': results_df.to_dict('records')
}

summary_path = RESULTS_DIR / "experiment_summary_all_tags.json"
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(experiment_summary, f, indent=2, ensure_ascii=False)

print(f"\nExperiment summary saved to: {summary_path}")

## Additional Analysis: Class Distribution

In [ ]:
# Visualize which tags were included
plt.figure(figsize=(12, 6))
tag_distribution = balanced_dataset['document_tag'].value_counts().sort_index()
plt.bar(range(len(tag_distribution)), tag_distribution.values)
plt.xticks(range(len(tag_distribution)), tag_distribution.index, rotation=45, ha='right')
plt.xlabel('Document Tag', fontsize=12, fontweight='bold')
plt.ylabel('Count', fontsize=12, fontweight='bold')
plt.title(f'Balanced Dataset Distribution ({NUM_TAGS} Classes)', fontsize=14, fontweight='bold')
plt.axhline(y=min_class_size, color='red', linestyle='--', label=f'Balanced count: {min_class_size}')
plt.legend()
plt.tight_layout()
dist_plot_path = RESULTS_DIR / "class_distribution.png"
plt.savefig(dist_plot_path, dpi=300, bbox_inches='tight')
print(f"Class distribution plot saved to: {dist_plot_path}")
plt.show()